# MOFs LM Probing for Topology

In [ ]:
from baseline import PromptSet

from tqdm.auto import tqdm
from nltk.corpus import stopwords

import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, logging

logging.set_verbosity_error()  # avoid irritating transformers warnings

## Read in the Triples

In [ ]:
file = "../data/mof_topology.csv"

In [ ]:
df = pd.read_csv(file)
df.head()

In [ ]:
df.shape

In [ ]:
df.ObjectEntity.nunique()

## Probing BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-large-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Create Prompts

In [ ]:
def create_topology_prompt1(sub, mask):
    return f"{sub} is an MOF. {sub} has the topology {mask}."
    
def create_topology_prompt2(sub, mask):
    return f"{sub} is an MOF. {sub} has an topology code {mask}."

def create_topology_prompt3(sub, mask):
    return f"{sub} is an MOF. {sub} has an tolology RCSR code {mask}."

def create_topology_prompt4(sub, mask):
    return f"{sub} is an MOF. The topology of {sub} is {mask}."
    
def create_topology_prompt5(sub, mask):
    return f"{sub} is an MOF. {sub} has the three letter symbol {mask}."
    
def create_topology_prompt6(sub, mask):
    return f"{sub} is an metal organic framework. {sub}'s topology is {mask}." 
    
def create_topology_prompt7(sub, mask):
    return f"{sub} is an MOF. {sub} is an type of {mask} topology."
    
def create_topology_prompt8(sub, mask):
    return f"{sub} is an MOF. {sub} is in the class of {mask} topology."
    
def create_topology_prompt9(sub, mask):
    return f"{sub} has SBUs and organic linkers. The topology of {sub} is {mask}."
    
def create_topology_prompt10(sub, mask):
    return f"{sub} is an MOF. The three letter {mask} describes {sub} topology."
    
def create_topology_prompt11(sub, mask):
    return f"{sub} is a MOF. The RCSR code of {sub} is {mask}."
    
def create_topology_prompt12(sub, mask):
    return f"As a MOF, {sub} has a topological structure {mask}."

In [ ]:
processors = [create_topology_prompt1, create_topology_prompt2,
              create_topology_prompt3, create_topology_prompt4,
              create_topology_prompt5, create_topology_prompt6,
              create_topology_prompt7, create_topology_prompt8,
              create_topology_prompt9, create_topology_prompt10,
              create_topology_prompt11, create_topology_prompt12
             ]

## Check Whether the Predicted Tokens Match

In [ ]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

## Test BERT

In [ ]:
def prompt_probe(model, tokenizer, prompt_processor, df_sub, mask_token, \
                 subjectCol, relationCol, objectCol, top_k=100):

    pipe = pipeline(
        task="fill-mask",
        model=model,
        tokenizer=tokenizer,
        top_k=top_k,
    )

    prompts = PromptSet([prompt_processor(row.SubjectEntity, mask_token) for _, row in df_sub.iterrows()])

    outputs = []
    for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
        outputs.append(out)

    results = []
    for idx, row in df_sub.iterrows():
        for output in outputs[idx]:
            record = {}
            record['SubjectEntity'] = row[subjectCol]
            record['Relation'] = row[relationCol]
            record['ObjectEntity'] = row[objectCol]
            record['prompt']= prompts[idx]
            record['predictedScore'] = output['score']
            record['predictedToken'] = output['token_str']
            results.append(record)

    return pd.DataFrame(results)

In [ ]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [ ]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [ ]:
# Assume the predicted_df has a binary column 'predicted' indicating whether ObjectEntity has been predicted
# 1 for yes, 0 for no
def evaluate_acc_n(predicted_df, relation, prompt_template, at_k=5):
    topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=at_k)
    topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')
    topk_agg_df.columns = ['SubjectEntity', 'predicted']
    result = {}
    result['Relation'] = relation
    result['prompt_template'] = prompt_template
    result['acc'] = topk_agg_df.predicted.mean()
    result['at_k'] = at_k

    return result

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasTopology', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasTopology_BERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasTopology_BERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing MatBERT

In [ ]:
# MatBERT
tokenizer = AutoTokenizer.from_pretrained("../model/model/matbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("../model/model/matbert-base-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasTopology', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasTopology_MatBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasTopology_MatBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing MatSciBERT

In [ ]:
# MatSciBERT
tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
model = AutoModelForMaskedLM.from_pretrained('m3rg-iitd/matscibert')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasTopology', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasTopology_MatSciBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasTopology_MatSciBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing SciBERT

In [ ]:
# SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModelForMaskedLM.from_pretrained('allenai/scibert_scivocab_cased')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasTopology', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasTopology_SciBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasTopology_SciBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing RoBERTa

In [ ]:
# RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasTopology', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasTopology_RoBERTa.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasTopology_RoBERTa_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)